# Assigment 4

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

In [6]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
from pathlib import Path
import glob
from itertools import chain
import shutil
from collections import Counter
pd.set_option('display.max_rows', None)

In this path **..\_data\sbs\B_RawData\bancos** you will find scraped data from [this link](https://www.sbs.gob.pe/app/pp/EstadisticasSAEEPortal/Paginas/TIActivaTipoCreditoEmpresa.aspx?tip=B). We get all the information of the last available day of every each month.

1. Import all the data and generate a column named as `date_info` that should have the day to information corresponds.

In [2]:
#First, we know that all the file are excel. 
files = os.listdir(r'../../_data/sbs/B_RawData/bancos/')   #Save in a list all the names of all the files         
y = [z[12:-5] for z in files] 

In [3]:
#This function is useful to rename duplicates columns. 
def df_column_uniquify(df):
    df_columns = df.columns
    new_columns = []
    for item in df_columns:
        counter = 0
        newitem = item
        while newitem in new_columns:
            counter += 1
            newitem = "{}_{}".format(item, counter)
        new_columns.append(newitem)
    df.columns = new_columns
    return df

In [7]:
li = {}    #Create an empty dictionary 
for f in y:
    li["table_"+f] = pd.read_excel(r'../../_data/sbs/B_RawData/bancos/table_clean_'+f+'.xlsx') #We save every data in li
    if datetime.strptime("31_8_2010", "%d_%m_%Y").date() >= datetime.strptime(f, "%d_%m_%Y").date() >= datetime.strptime("30_9_2002", "%d_%m_%Y").date():   #For the datas in these year need transponse
            li["table_"+f] = li["table_"+f].iloc[1:].T.reset_index().set_axis(li["table_"+f].iloc[1:].T.reset_index().iloc[0], axis=1).iloc[1:]
            li["table_"+f]["date_info"] = datetime.strptime(f, "%d_%m_%Y").date() #We add the column "date_info" in every dataframe
            df_column_uniquify(li["table_"+f])
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[0]:'TIPO', li["table_"+f].columns[1]:'TASA ANUAL (%)', 'Promedio del Sistema':'PROMEDIO'}) #Transponse
    
    if datetime.strptime("31_8_2010", "%d_%m_%Y").date() >= datetime.strptime(f, "%d_%m_%Y").date() >= datetime.strptime("30_5_2008", "%d_%m_%Y").date(): 
               
            li["table_"+f] = li["table_"+f].dropna(axis=0, how = 'all', subset=(li["table_"+f].columns[2], )) #We drop the empty rows
            a = li["table_"+f][li["table_"+f].iloc[:,2].apply(lambda c: sum([Counter(c.lower()).get(i, 0)   #Separate the data in two subdatas. In the first we shift the info
                                   for i in list('aeiou')]) >= 1)].shift(-1, axis= 'columns')
            b =  li["table_"+f][li["table_"+f].iloc[:,2].apply(lambda c: sum([Counter(c.lower()).get(i, 0)  
                                   for i in list('aeiou')]) == 0)]
            li["table_"+f] = b.append(a)                                                                  #append again 
            li["table_"+f]["date_info"]  = np.where(li["table_"+f][np.nan].isna() == False,            
                                             li["table_"+f][np.nan], li["table_"+f]["date_info"] ) 
            li["table_"+f].drop(columns = [np.nan], axis = 1, inplace = True)                             #drop tha column np.nan
                
  
    else: 
            li["table_"+f] = li["table_"+f].rename(columns=str.upper)
            li["table_"+f] = li["table_"+f][~li["table_"+f]["TASA ANUAL (%)"].str.contains('|'.join(["Nota"]))]  #Drop row that has the word "Nota"
            li["table_"+f] = li["table_"+f].rename(columns={"DATE_INFO":'date_info'})
            li["table_"+f]["date_info"] = datetime.strptime(f, "%d_%m_%Y").date()
        

2. Append all this datasets and generate a unique dataframe. This new dataset should have information at `rate interest` and `date` level. The columns should be the name of the banks. Be careful since not all the excel files have the same format. **It is totally prohibited to manipulate manually the excel files. This kind of action means NO GRADE on this project.**

In [8]:
#Create a new dictonary only with the types of rates 
holi = {}
for f in y:
    if datetime.strptime("31_8_2010", "%d_%m_%Y").date() >= datetime.strptime(f, "%d_%m_%Y").date() >= datetime.strptime("30_9_2002", "%d_%m_%Y").date():
        holi["tipo_"+f] = li["table_"+f][['TIPO','TASA ANUAL (%)','date_info']]
        li["table_"+f] = li["table_"+f].drop(li["table_"+f].columns[0], axis = 1)
        

In [9]:
#APPEND THE DATA WITH THE TYPES 
df3 = pd.concat(holi.values(), axis=0).reset_index(drop=True).sort_values(by=['date_info']).drop_duplicates(subset=['TASA ANUAL (%)'])
len(df3)

35

In [10]:
data = [(li["table_"+x].columns.values.tolist(), li["table_"+x].iloc[0,-1]) for x in y] # to use the second column and "date" of "li" and create a tuple of the two values to append it  
data.sort(key=lambda x: x[1])  # to order the data by the date

for col, date in data: # loop for generate col and date 
    print(f'Columns: [{col}], date: [{date}]') # print the result

Columns: [['TASA ANUAL (%)', 'BBVA', 'COMERCIO', 'CRÉDITO', 'PICHINCHA', 'BIF', 'SCOTIABANK', 'CITIBANK', 'INTERBANK', 'MIBANCO', 'GNB', 'FALABELLA', 'SANTANDER', 'RIPLEY', 'ALFIN', 'ICBC', 'BANK OF CHINA', 'PROMEDIO', 'date_info']], date: [2001-03-31]
Columns: [['TASA ANUAL (%)', 'TRABAJO', 'BBVA', 'BANCO DE COMERCIO', 'BANCO DE CREDITO', 'BANCO PICHINCHA', 'BANBIF', 'B SANTANDER CENTRAL', 'STANDARD CHARTERED', 'BANCO SUDAMERICANO', 'WIESE SUDAMERIS', 'BANK BOSTON', 'BNP PARIBAS EL', 'CITIBANK DEL PERU', 'INTERBANK', 'MIBANCO', 'PROMEDIO', 'date_info']], date: [2002-09-30]
Columns: [['TASA ANUAL (%)', 'TRABAJO', 'BBVA', 'BANCO DE COMERCIO', 'BANCO DE CREDITO', 'BANCO PICHINCHA', 'BANBIF', 'B SANTANDER CENTRAL', 'STANDARD CHARTERED', 'BANCO SUDAMERICANO', 'WIESE SUDAMERIS', 'BANK BOSTON', 'BNP PARIBAS EL', 'CITIBANK DEL PERU', 'INTERBANK', 'MIBANCO', 'PROMEDIO', 'date_info']], date: [2002-10-31]
Columns: [['TASA ANUAL (%)', 'TRABAJO', 'BBVA', 'BANCO DE COMERCIO', 'BANCO DE CREDITO', 'B

In [11]:
#We made a loop to rename the names of banks 
for f in y:
    for x in range(len(li["table_"+f].axes[1])):
        if li["table_"+f].columns[x] in ('FINANCIERO', 'BANCO PICHINCHA'):  
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'PICHINCHA'})
        if li["table_"+f].columns[x] in ('BBVA', 'CONTINENTAL'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'BBVA'})
        if li["table_"+f].columns[x] in ('BANCO DE COMERCIO'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'COMERCIO'})
        if li["table_"+f].columns[x] in ('B SANTANDER CENTRAL'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'SANTANDER'})
        if li["table_"+f].columns[x] in ('AZTECA'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'ALFIN'})
        if li["table_"+f].columns[x] in ('SCOTIABANK PERU', 'BANCO SUDAMERICANO'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'SCOTIABANK'})
        if li["table_"+f].columns[x] in ('BANCO GNB', 'HSBC'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'GNB'})
        if li["table_"+f].columns[x] in ('BANCO DE CREDITO'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'CRÉDITO'})
        if li["table_"+f].columns[x] in ('CITIBANK DEL PERU'):
            li["table_"+f] = li["table_"+f].rename(columns = {li["table_"+f].columns[x]:'CITIBANK'})
       

In [12]:
#We check the changes 
li["table_30_12_2009"]

,TASA ANUAL (%),BBVA,COMERCIO,CRÉDITO,PICHINCHA,BANBIF,SCOTIABANK,CITIBANK,INTERBANK,MIBANCO,GNB,BANCO FALABELLA,SANTANDER PERU,BANCO RIPLEY,ALFIN BANCO,DEUTSCHE BANK PERU,PROMEDIO,date_info
1,Avances en Cuenta Corriente,16.05,-,41.05,-,25.57,6.47,4.30,5.30,-,-,-,-,-,-,-,23.56,2009-12-30
2,Sobregiros,48.68,138.18,78.72,-,82.42,51.97,10.01,12.57,-,60.00,-,-,-,-,-,64.60,2009-12-30
3,Descuentos,14.75,15.66,6.03,9.09,7.01,7.26,-,7.10,-,6.50,-,-,-,-,-,7.66,2009-12-30
4,Clientes deuda > S/. 5 MM4,3.82,5.50,3.12,12.17,5.53,4.12,4.64,5.29,-,6.38,-,-,-,-,-,3.63,2009-12-30
5,Demás Clientes,9.34,16.39,4.96,17.67,7.99,5.74,3.92,5.56,24.57,5.64,2.25,5.71,-,-,-,7.37,2009-12-30
6,Promedio,6.39,14.45,3.84,14.58,6.75,5.30,4.55,5.49,24.57,6.10,2.25,5.71,-,-,-,5.32,2009-12-30
7,Clientes deuda > S/. 5 MM,1.82,-,2.15,-,-,-,4.02,6.59,-,2.95,-,-,-,-,-,2.05,2009-12-30
8,Demás Clientes,9.65,13.10,5.14,13.00,2.85,2.96,-,8.30,-,-,-,2.39,-,-,-,5.91,2009-12-30
9,Promedio,3.00,13.10,2.83,13.00,2.85,2.96,4.02,7.21,-,2.95,-,2.39,-,-,-,3.04,2009-12-30
10,Clientes deuda > S/. 5 MM,3.47,-,3.07,8.87,5.93,3.04,4.49,3.88,-,3.53,-,-,-,-,-,3.41,2009-12-30


In [13]:
#APPEND THE DATA 
DATA = pd.concat(li.values(), axis=0).reset_index(drop=True).sort_values(by=['date_info']).replace("-",np.nan)

3. What are the top 5 banks each year with the highest interest rate at `Préstamos hipotecarios para vivienda`, `Consumo -
Tarjetas de Crédito`. Present a dataframe with these variables: `year`, `rate_concept`, `banks`, `rate_value`.

In [14]:
DATA["TASA ANUAL (%)"].unique()

array(['Préstamos a más de 360 días', 'Préstamos de 91 a 180 días',
       'Préstamos de 181 a 360 días', 'Microempresas',
       'Tarjetas de Crédito', 'Descuentos', 'Préstamos Revolventes',
       'Préstamos a cuota fija de 31 a 90 días',
       'Préstamos a cuota fija de 91 a 180 días',
       'Préstamos de 31 a 90 días',
       'Préstamos a cuota fija de 181 a 360 días', 'Consumo',
       'Préstamos no Revolventes para automóviles',
       'Préstamos no Revolventes para libre disponibilidad hasta 360 días',
       'Préstamos no Revolventes para libre disponibilidad a más de 360 días',
       'Créditos pignoraticios', 'Hipotecarios',
       'Préstamos hipotecarios para vivienda',
       'Préstamos a cuota fija a más de 360 días',
       'Préstamos hasta 30 días', 'Préstamos a cuota fija hasta 30 días',
       'Pequeñas Empresas', 'Corporativos', 'Grandes Empresas',
       'Medianas Empresas', 'Dsctos. y préstamos hasta 360 días',
       'Dsctos. y préstamos hasta 30 días',
       'D

In [15]:
df3  #See what type of credit card is CONSUMO

,TIPO,TASA ANUAL (%),date_info
1318,CONSUMO,Tarjetas de crédito abiertas4,2002-09-30
1308,COMERCIAL,Avances en Cta.Corriente,2002-09-30
1309,COMERCIAL.1,Sobregiros,2002-09-30
1310,COMERCIAL.2,Dsctos. y préstamos hasta 30 días,2002-09-30
1311,COMERCIAL.3,Dsctos. y préstamos 31 - 90 días,2002-09-30
1312,COMERCIAL.4,Dsctos. y préstamos 91 - 180 días,2002-09-30
1313,COMERCIAL.5,Dsctos. y préstamos 181 - 360 días,2002-09-30
1314,COMERCIAL.6,Dsctos. y préstamos más 360 días,2002-09-30
1316,A MICROEMPRESAS2.1,Dsctos. y préstamos hasta 360 días,2002-09-30
1319,CONSUMO.1,Tarjetas de crédito cerradas5,2002-09-30


In [16]:
#Homogenize the values
DATA["TASA ANUAL (%)"] = DATA["TASA ANUAL (%)"]. replace([ "Tarjetas de crédito", "Tarjetas de crédito abiertas4",  "Tarjetas de crédito cerradas4",
                                                          "Tarjetas de crédito cerradas5", "Tarjetas de crédito cerradas", "Tarjetas de crédito abiertas3" ], "Tarjetas de Crédito")
DATA["TASA ANUAL (%)"] = DATA["TASA ANUAL (%)"]. replace([ "Hipotecarios", "HIPOTECARIO", 
                                                          "PRÉSTAMOS HIPOTECARIOS" ], "Préstamos hipotecarios para vivienda")

#Keep with the values indicates
TOP = DATA[DATA["TASA ANUAL (%)"].isin(["Tarjetas de Crédito", "Préstamos hipotecarios para vivienda"])]

#Create a variable: year
TOP["date_info"] = pd.to_datetime(TOP.date_info, format='%Y-%m-%d')
TOP['year'] = TOP['date_info'].dt.year

#Drop unnecessary variables
TOP = TOP.drop(columns=['PROMEDIO', 'date_info'])

#Format wide to long 
TOP = TOP.melt(id_vars = ["year", "TASA ANUAL (%)"], var_name= 'banks',value_name= 'rate_value').dropna(subset=['rate_value'], axis = 'rows' )
TOP = TOP[TOP.rate_value != 's.i.']  #We have banks without information. Is not necessary for us. 

#Create the dataframe with the top 5 rates for year and for concept. 
TOP5= TOP.sort_values("rate_value", ascending = False).groupby(["year","TASA ANUAL (%)"]).head(5).reset_index(drop = True).sort_values(["year", "TASA ANUAL (%)", "rate_value"],
                                                                                                                                 ascending = [True, False, False]).rename(columns={'TASA ANUAL (%)': 'rate_concept'})

C:\Users\Grade\AppData\Local\Temp/ipykernel_14056/2639930001.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TOP["date_info"] = pd.to_datetime(TOP.date_info, format='%Y-%m-%d')
C:\Users\Grade\AppData\Local\Temp/ipykernel_14056/2639930001.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TOP['year'] = TOP['date_info'].dt.year


In [17]:
TOP5

,year,rate_concept,banks,rate_value
177,2001,Tarjetas de Crédito,RIPLEY,72.02
179,2001,Tarjetas de Crédito,FALABELLA,71.82
203,2001,Tarjetas de Crédito,INTERBANK,57.73
207,2001,Tarjetas de Crédito,BBVA,47.30
208,2001,Tarjetas de Crédito,GNB,40.40
153,2001,Préstamos hipotecarios para vivienda,COMERCIO,8.24
154,2001,Préstamos hipotecarios para vivienda,COMERCIO,8.24
155,2001,Préstamos hipotecarios para vivienda,PICHINCHA,8.20
156,2001,Préstamos hipotecarios para vivienda,PICHINCHA,8.20
186,2001,Préstamos hipotecarios para vivienda,GNB,7.69


4. We want to save this dataset in the folder **_output/sbs/group_#**, but we want to save a file per bank. We want to have the information disaggregated at the bank level. Please, save your files with the name of the bank. Avoid blank spaces and use only lowercase letters. Generate the folder of your group using python code. **Hint: os library**

In [18]:
# Python program to explain os.getcwd() method

# Get the current working
# directory (CWD)
cwd = os.getcwd()
# Print the current working
# directory (CWD)
print("Current working directory:", cwd)

Current working directory: C:\Users\Grade\Dropbox\GitHub\Diplomado_PUCP\Lecture_4\Assigment_4


In [19]:
# Directory 
directory = "group6"
  
# Parent Directory path 
parent_dir = "../../_output/sbs/"
# Path 
path = os.path.join(parent_dir, directory) 

#Create the directory
if not os.path.exists(path):
    os.makedirs(path)
else:
    shutil.rmtree(path)           # Removes all the subdirectories!
    os.makedirs(path)
    

print("Directory '% s' created" % directory) 

Directory 'group6' created


In [20]:
#Create a list with the unique names of the banks
BANKS = list(TOP5.banks.unique())

# Loop for each bank in df
for i in BANKS:
    TOP5.loc[TOP5["banks"] == i,:].to_excel(path+"/"+i.lower() +".xlsx")